In [ ]:
!pip install gensim --quiet
!pip install tensorflow-text==2.15.0 --quiet #15
!pip install tensorflow==2.15.0 --quiet #15 13
!pip install tf_keras==2.15.0 --quiet
!pip install transformers==4.17 --quiet
!pip install pydot --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 36.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text
from sklearn.model_selection import train_test_split


import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re


import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
wiki_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/wiki_movie_plots_deduped.csv")
import glob
import pandas as pd

# get data file names
path =r'/content/drive/My Drive/Colab Notebooks/movie genres'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

genre_dataset = pd.concat(li, axis=0, ignore_index=True)
genre_dataset.drop_duplicates(subset="movie_name", keep='first', inplace=True)

In [ ]:
movie_genre_data = pd.merge(genre_dataset, wiki_data, left_on='movie_name', right_on='Title')

In [ ]:
# prompt: how many distinct genres are there in the genre column, which gives multple genres

# Extract unique genres from the 'Genre' column
unique_genres = set()
for index, row in movie_genre_data.iterrows():
    genres = str(row['genre']).split(', ')
    for genre in genres:
        unique_genres.add(genre.strip())

# Count the number of distinct genres
num_distinct_genres = len(unique_genres)
print(f"There are {num_distinct_genres} distinct genres in the 'genre' column.")


There are 24 distinct genres in the 'genre' column.


In [ ]:
# prompt: remove rows that include genres with very low count (below 30)
movie_genre_data['genre'].str.split(', ', expand=True).stack().value_counts()

Drama         11818
Action         7013
Comedy         6590
Romance        5697
Crime          5291
Adventure      4051
Thriller       3200
Horror         2104
Mystery        1905
Family         1591
Fantasy        1317
Sci-Fi         1141
Biography       925
Music           795
Musical         771
War             733
History         704
Animation       658
Film-Noir       607
Sport           420
Western         301
Game-Show         1
Adult             1
Reality-TV        1
Name: count, dtype: int64

In [ ]:
# prompt: drop rows with genre which include Game-Show , Adult, Reality-TV

movie_genre_data = movie_genre_data[~movie_genre_data['genre'].str.contains('Game-Show')]
movie_genre_data = movie_genre_data[~movie_genre_data['genre'].str.contains('Adult')]
movie_genre_data = movie_genre_data[~movie_genre_data['genre'].str.contains('Reality-TV')]


In [ ]:
# prompt: create a new dataframe with columns movie_name, plot, and a 1 hot encoding of all the genres

import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Extract genres and convert to a list of lists
genres = movie_genre_data['genre'].str.split(', ').tolist()

# Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Fit and transform genres to one-hot encoding
genre_onehot = mlb.fit_transform(genres)

# Create a new DataFrame with one-hot encoded genres
genre_df = pd.DataFrame(genre_onehot, columns=mlb.classes_)

# Concatenate movie_name, plot, and genre_onehot DataFrames
new_df = pd.concat([movie_genre_data[['Plot']], genre_df], axis=1)

# Display the new DataFrame
print(new_df.head())


                                                Plot  Action  Adventure  \
0  In 2154, humans have depleted Earth's natural ...     1.0        1.0   
1  Albus Dumbledore, Minerva McGonagall, and Rube...     0.0        1.0   
2  In the mid-21st century, crop blights and dust...     0.0        1.0   
3  In the mid-21st century, crop blights and dust...     0.0        1.0   
4  As punishment for a past rebellion, the 12 dis...     1.0        1.0   

   Animation  Biography  Comedy  Crime  Drama  Family  Fantasy  ...  Horror  \
0        0.0        0.0     0.0    0.0    0.0     0.0      1.0  ...     0.0   
1        0.0        0.0     0.0    0.0    0.0     1.0      1.0  ...     0.0   
2        0.0        0.0     0.0    0.0    1.0     0.0      0.0  ...     0.0   
3        0.0        0.0     0.0    0.0    1.0     0.0      0.0  ...     0.0   
4        0.0        0.0     0.0    0.0    0.0     0.0      0.0  ...     0.0   

   Music  Musical  Mystery  Romance  Sci-Fi  Sport  Thriller  War  Western

In [ ]:
new_df.columns

Index(['Plot', 'Action', 'Adventure', 'Animation', 'Biography', 'Comedy',
       'Crime', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror',
       'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller',
       'War', 'Western'],
      dtype='object')

In [ ]:
# prompt: remove all rows where the plot is not a string

# Check the data types of the 'Plot' column
print(new_df['Plot'].apply(type).value_counts())

# Filter rows where 'Plot' is not a string
new_df = new_df[new_df['Plot'].apply(type) == str]

# Check the data types of the 'Plot' column after filtering
print(new_df['Plot'].apply(type).value_counts())


Plot
<class 'str'>      22829
<class 'float'>        3
Name: count, dtype: int64
Plot
<class 'str'>    22829
Name: count, dtype: int64


In [ ]:
# prompt: create xtrain, ytrain, xtest, and ytest for this dataset, with the x being the plot, and y being a one hot encoding vector

from sklearn.model_selection import train_test_split

# Split the dataset into features (X) and labels (y)
X = new_df['Plot']
y = new_df.drop(['Plot'], axis=1)

# Split the data into training and testing sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (18263,)
y_train shape: (18263, 21)
X_test shape: (4566,)
y_test shape: (4566, 21)


In [ ]:
y.iloc[0]

Action       1.0
Adventure    1.0
Animation    0.0
Biography    0.0
Comedy       0.0
Crime        0.0
Drama        0.0
Family       0.0
Fantasy      1.0
Film-Noir    0.0
History      0.0
Horror       0.0
Music        0.0
Musical      0.0
Mystery      0.0
Romance      0.0
Sci-Fi       0.0
Sport        0.0
Thriller     0.0
War          0.0
Western      0.0
Name: 0, dtype: float64

In [ ]:
# prompt: convert the data to tensorflow datasets
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))


In [ ]:
train_examples, train_labels = next(iter(train_data.batch(20000)))
test_examples, test_labels = next(iter(test_data.batch(5000)))


In [ ]:
tokenizer = tf_text.WhitespaceTokenizer()
train_tokens = tokenizer.tokenize(train_examples)
test_tokens = tokenizer.tokenize(test_examples)

In [ ]:
MAX_SEQUENCE_LENGTH = 500
NUM_CLASSES = 21

In [ ]:
nltk.download('word2vec_sample')
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
vec_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.


In [ ]:
EMBEDDING_DIM = 300

# initialize embedding matrix and word-to-id map:
embedding_matrix = np.zeros((len(vec_model) + 1, EMBEDDING_DIM))
vocab_dict = {}

# build the embedding matrix and the word-to-id map:
for i, word in enumerate(vec_model.index_to_key):
    embedding_vector = vec_model[word]

    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        vocab_dict[word] = i

# we can use the last index at the end of the vocab for unknown tokens
vocab_dict['[UNK]'] = len(vocab_dict)

In [ ]:
def docs_to_vocab_ids(tokenized_texts_list, max_sequence_length = MAX_SEQUENCE_LENGTH):
    """
    converting a list of strings to a list of lists of word ids
    """
    texts_vocab_ids = []
    for i, token_list in enumerate(tokenized_texts_list):

        # Get the vocab id for each token in this doc ([UNK] if not in vocab)
        vocab_ids = []
        for token in list(token_list.numpy()):
            decoded = token.decode('utf-8', errors='ignore')
            if decoded in vocab_dict:
                vocab_ids.append(vocab_dict[decoded])
            else:
                vocab_ids.append(vocab_dict['[UNK]'])

        # Truncate text to max length, add padding up to max length
        vocab_ids = vocab_ids[:max_sequence_length]
        n_padding = (max_sequence_length - len(vocab_ids))
        # For simplicity in this model, we'll just pad with uknown tokens
        vocab_ids += [vocab_dict['[UNK]']] * n_padding

        # Add this example to the list of converted docs
        texts_vocab_ids.append(vocab_ids)

        if i % 5000 == 0:
            print('Examples processed: ', i)

    print('Total examples: ', i)

    return np.array(texts_vocab_ids)

In [ ]:
train_input = docs_to_vocab_ids(train_tokens)
test_input = docs_to_vocab_ids(test_tokens)

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

Examples processed:  0
Examples processed:  5000
Examples processed:  10000
Examples processed:  15000
Total examples:  18262
Examples processed:  0
Total examples:  4565


## MultiLabel Bert multiclass classification

In [ ]:
from transformers import BertTokenizer, TFBertModel

checkpoint = 'bert-base-cased'
bert_tokenizer = BertTokenizer.from_pretrained(checkpoint)
bert_model = TFBertModel.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
def create_bert_multilabel_model(checkpoint = checkpoint,
                                 num_classes = 21,
                                 hidden_size=768,
                                 dropout=0.3,
                                 learning_rate=0.00005):
    """
    Build a multi-label classification model with BERT. Use the Pooler Output for classification purposes.
    """
    # Load the BERT model
    bert_model = TFBertModel.from_pretrained(checkpoint)

    # Make all BERT layers trainable
    for layer in bert_model.layers:
        layer.trainable = True

    # Define input layers
    input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(None,), dtype=tf.int64, name='attention_mask_layer')

    # Prepare BERT inputs
    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    # Get BERT outputs
    bert_out = bert_model(bert_inputs)
    pooler_token = bert_out[1]

    # Add hidden layer with dropout
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooler_token)
    hidden = tf.keras.layers.Dropout(dropout)(hidden)

    # Add output layer for multi-label classification
    classification = tf.keras.layers.Dense(num_classes, activation='sigmoid', name='classification_layer')(hidden)

    # Define the model
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

    # Compile the model with appropriate loss function and metrics
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=Custom_Hamming_Loss(from_logits=False),
                                 metrics=['accuracy'])

    return classification_model

In [ ]:
train_texts_bert = X_train.tolist()
test_texts_bert = X_test.tolist()
valid_texts_bert = train_texts_bert[16000:]
valid_labels_bert = y_train[16000:]
train_texts_bert = train_texts_bert[:16000]
train_labels_bert = y_train[:16000]
test_labels_bert = y_test
npvalid_labels = np.asarray(valid_labels_bert)

In [ ]:
def create_encodings_with_max_length(max_length):
  train_encodings = bert_tokenizer(train_texts_bert, truncation=True, padding=True, max_length=max_length, return_tensors='tf')
  valid_encodings = bert_tokenizer(valid_texts_bert, truncation=True, padding=True, max_length=max_length, return_tensors='tf')
  test_encodings = bert_tokenizer(test_texts_bert, truncation=True, padding=True, max_length=max_length, return_tensors='tf')

  return train_encodings, valid_encodings, test_encodings

In [ ]:
train_encodings, valid_encodings, test_encodings = create_encodings_with_max_length(100)
pooler_bert_model = create_bert_multilabel_model()

pooler_bert_model_history = pooler_bert_model.fit([train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask],
                                                  train_labels_bert,
                                                  validation_data=([valid_encodings.input_ids, valid_encodings.token_type_ids, valid_encodings.attention_mask],
                                                  npvalid_labels),
                                                  batch_size=8,
                                                  epochs=3)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/3
2000/2000 [==============================] - 424s 193ms/step - loss: 0.2839 - accuracy: 0.2557 - val_loss: nan - val_accuracy: 0.2872
Epoch 2/3
2000/2000 [==============================] - 376s 188ms/step - loss: 0.2520 - accuracy: 0.3148 - val_loss: nan - val_accuracy: 0.2943
Epoch 3/3
2000/2000 [==============================] - 379s 189ms/step - loss: 0.2233 - accuracy: 0.3568 - val_loss: nan - val_accuracy: 0.3067


In [ ]:
predictions = pooler_bert_model.predict([test_encodings.input_ids, test_encodings.token_type_ids, test_encodings.attention_mask])


143/143 [==============================] - 33s 214ms/step


In [ ]:
def Custom_Hamming_Loss(y_true, y_pred):
  return K.mean(y_true*(1-y_pred)+(1-y_true)*y_pred)